In [100]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Leitura de instâncias

In [101]:
def read_dat_file(file_path):
    """"Função para a leitura das instâncias de Mariá"""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # 1. Lendo quantidade de itens e períodos
    items, periods = map(int, lines[0].split())

    # 2. Lendo número de plantas
    num_plants = int(lines[1].strip())

    # 3. Lendo capacidades das plantas
    capacities = [int(lines[i + 2].strip()) for i in range(num_plants)]
    capacities = np.tile(capacities, (periods, 1)).T  # Repete as capacidades ao longo dos períodos (deixar na forma j, t)

    # 4. Lendo a matriz de produção (tempo de produção, tempo de setup, custo de setup, custo de produção)
    production_data = []
    start_line = 2 + num_plants
    production_time = np.zeros((items, num_plants))  # Inicializar listas para armazenar separadamente os tempos e custos
    setup_time = np.zeros((items, num_plants))
    setup_cost = np.zeros((items, num_plants))
    production_cost = np.zeros((items, num_plants))
    for i in range(num_plants * items):  # Preencher as matrizes com os dados lidos
        plant = i // items  # Determina a planta
        item = i % items    # Determina o item
        # Extrair os dados de cada linha
        prod_time, set_time, set_cost, prod_cost = map(float, lines[start_line + i].split())
        production_time[item, plant] = prod_time  # Preencher as respectivas matrizes
        setup_time[item, plant] = set_time
        setup_cost[item, plant] = set_cost
        production_cost[item, plant] = prod_cost

    # 5. Lendo os custos de inventário
    inventory_costs_line = start_line + num_plants * items
    inventory_costs = list(map(float, lines[inventory_costs_line].split()))  # Lê todos os valores de inventory_costs como uma única lista
    inventory_costs = np.array(inventory_costs).reshape(num_plants, -1)  # Divide a lista de custos de inventário por planta
    inventory_costs = inventory_costs.T  # Deixa na forma (i, j)

    # 6. Lendo a matriz de demanda (12 linhas, 12 colunas)
    demand_matrix = []
    demand_start_line = inventory_costs_line + 1

    # Verifica se a matriz de demanda está na forma padrão
    num_demand_lines = 0
    for i in range(demand_start_line, len(lines)):
        if lines[i].strip():  # Se a linha não estiver vazia, contamos como linha de demanda
            num_demand_lines += 1
        else:
            break  # Interrompe se encontrar uma linha vazia (ou um bloco separado)
    
    # Calcula o multiplicador, caso tenha mais linhas do que períodos
    if num_demand_lines == periods * 2:
        multiplier = 2  # A matriz de demanda tem o dobro de linhas
    else:
        multiplier = 1  # A matriz de demanda tem o número esperado de linhas (T)
    
    # Leitura inicial das demandas
    for i in range(periods * multiplier):  # Lê as linhas de demandas para os períodos
        demands = list(map(int, lines[demand_start_line + i].split()))
        demand_matrix.append(demands)

    # print('Linha 1:', demand_matrix[0])
    # print('Linha 13:', demand_matrix[periods], '\n\n')
    
    # Agora, se multiplier == 2, precisamos combinar as linhas 1-12 com 13-24 corretamente
    if multiplier == 2:
        new_demand_matrix = []
        
        for i in range(periods):  # Para cada linha do primeiro bloco (1-12)
            combined_period_demand = []  # Lista única para armazenar a demanda combinada para o período i
            
            # Quebrar a linha i entre as plantas
            demands_first_part = demand_matrix[i]
            demands_second_part = demand_matrix[i + periods]  # Linha correspondente do segundo bloco
            
            # Calcular o número de elementos por planta
            elements_per_plant_first = len(demands_first_part) // num_plants
            elements_per_plant_second = len(demands_second_part) // num_plants
            
            # Concatenar as demandas para cada planta
            for j in range(num_plants):
                # Particionar a demanda de cada planta
                plant_demand_first = demands_first_part[j*elements_per_plant_first:(j+1)*elements_per_plant_first]
                plant_demand_second = demands_second_part[j*elements_per_plant_second:(j+1)*elements_per_plant_second]
                
                # Concatenar as demandas da planta j (primeiro e segundo blocos) e adicionar diretamente à lista única
                combined_period_demand.extend(plant_demand_first + plant_demand_second)
            
            # Adicionar a demanda ajustada (em formato de lista única) do período à nova matriz de demanda
            new_demand_matrix.append(combined_period_demand)
        
        # Substitui a matriz de demandas pela combinada e particionada por planta
        demand_matrix = new_demand_matrix
    
    # print('Linha 1 output após junção:', demand_matrix[0])

    # Agora vamos dividir os valores de cada linha combinada entre as plantas
    final_demand_matrix = []
    for demands in demand_matrix:
        period_demand = []
        for j in range(num_plants):
            # Divide a demanda combinada por planta, assumindo que cada planta tem o mesmo número de itens
            plant_demand = demands[j*items:(j+1)*items]
            period_demand.append(plant_demand)
        final_demand_matrix.append(period_demand)
    
    # Transpor a matriz de demanda para o formato correto (itens, plantas, períodos)
    final_demand_matrix = np.array(final_demand_matrix)
    # print(final_demand_matrix)
    final_demand_matrix = np.transpose(final_demand_matrix, (2, 1, 0))  # Converte para o formato (itens, plantas, períodos)

    # 7. Lendo os custos de transferência
    transfer_costs = []
    transfer_cost_line = demand_start_line + periods * multiplier
    while transfer_cost_line < len(lines):
        line = lines[transfer_cost_line].strip()  # Verificar se a linha não está vazia antes de tentar ler
        if line:
            transfer_costs.append(float(line))
        transfer_cost_line += 1

    def create_transfer_cost_matrix(transfer_costs, num_plants):  # Criar a matriz de custos de transferência (simétrica)
        transfer_cost_matrix = np.zeros((num_plants, num_plants))  # Inicializar a matriz de zeros
        if len(transfer_costs) == 1:
            transfer_cost = transfer_costs[0]  # Se houver apenas um custo de transferência, aplicar para todos os pares de plantas
            for j in range(num_plants):
                for k in range(j + 1, num_plants):
                    transfer_cost_matrix[j, k] = transfer_cost
                    transfer_cost_matrix[k, j] = transfer_cost
        else:
            idx = 0  # Se houver múltiplos custos, aplicar entre pares de plantas
            for j in range(num_plants):
                for k in range(j + 1, num_plants):
                    transfer_cost_matrix[j, k] = transfer_costs[idx]
                    transfer_cost_matrix[k, j] = transfer_costs[idx]
                    idx += 1
        return transfer_cost_matrix

    transfer_costs = create_transfer_cost_matrix(transfer_costs, num_plants)

    return {"items": items,
            "periods": periods,
            "num_plants": num_plants,
            "capacities": capacities,
            "production_time": production_time,
            "setup_time": setup_time,
            "setup_cost": setup_cost,  
            "production_cost": production_cost,
            "inventory_costs": inventory_costs,
            "demand_matrix": final_demand_matrix,
            "transfer_costs": transfer_costs}

In [102]:
# Exemplo de uso
file_path = '../instancias/maria_desiree/AAA01246_0.dat'
data = read_dat_file(file_path)
display(data)

{'items': 6,
 'periods': 12,
 'num_plants': 4,
 'capacities': array([[1623, 1623, 1623, 1623, 1623, 1623, 1623, 1623, 1623, 1623, 1623,
         1623],
        [1517, 1517, 1517, 1517, 1517, 1517, 1517, 1517, 1517, 1517, 1517,
         1517],
        [1925, 1925, 1925, 1925, 1925, 1925, 1925, 1925, 1925, 1925, 1925,
         1925],
        [1870, 1870, 1870, 1870, 1870, 1870, 1870, 1870, 1870, 1870, 1870,
         1870]]),
 'production_time': array([[2.2, 3.2, 4.4, 3.6],
        [1.6, 1.9, 3.8, 4.6],
        [4.7, 4.7, 4.2, 2.2],
        [4.9, 1.1, 2.7, 1.9],
        [3.1, 1.5, 4.2, 1.6],
        [2.7, 3.8, 2.7, 3.2]]),
 'setup_time': array([[38.6, 16.8, 48.9, 60.8],
        [58.1, 67.8, 71.5, 51.6],
        [57.6, 65.5, 22.4, 68.5],
        [20.6, 44.8, 66.7, 71.9],
        [16. , 18. , 15.5, 53.4],
        [73.3, 23. , 56.7, 55.9]]),
 'setup_cost': array([[ 78.1, 870.5, 819.8, 240.3],
        [241.2, 185. , 601.2, 139.1],
        [938.2, 898.8, 506.7, 918.5],
        [375.4, 643.1, 2

# Modelagem

In [103]:
m = gp.Model('Lot-sizing Sambasivan and Yahya')

## Conjuntos

In [104]:
# Produtos (i)
I = np.array([_ for _ in range(data['items'])])
# Plantas (j)
J = np.array([_ for _ in range(data['num_plants'])])
# Períodos (t)
T = np.array([_ for _ in range(data['periods'])])

## Parâmetros

In [105]:
r

array([[0.  , 0.25, 0.28, 0.28],
       [0.25, 0.  , 0.25, 0.25],
       [0.28, 0.25, 0.  , 0.39],
       [0.28, 0.25, 0.39, 0.  ]])

In [106]:
# Demanda (i, j, t)
d = np.array(data['demand_matrix'])
# Capacidade (j, t)
cap = np.array(data['capacities'])
# Tempo de produção (i, j)
b = np.array(data['production_time'])
# Tempo de setup (i, j)
f = np.array(data['setup_time'])
# Custo de produção (i, j)
c = np.array(data['production_cost'])
# Custo de setup (i, j)
s = np.array(data['setup_cost'])
# Custo de transporte (j, k)
r = np.array(data['transfer_costs'])
# Custo de estoque (i, j)
h = np.array(data['inventory_costs'])

## Variáveis de decisão

In [107]:
# Quantidade produzida (i, j, t)
X = m.addVars(I, J, T, vtype=GRB.INTEGER, name='X')
# Quantidade estocada (i, j, t)
Q = m.addVars(I, J, T, vtype=GRB.INTEGER, name='Q')
# # Quantidade transportada (i, j, k(um outro j), t)
W = m.addVars(I, J, J, T, vtype=GRB.CONTINUOUS, name='W')
# # Variável de setup (binária) (i, j, t)
Z = m.addVars(I, J, T, vtype=GRB.BINARY, name='Z')

## Função objetivo

In [108]:
expr_objetivo = sum(sum(sum(c[i, j] * X[i, j, t] + h[i, j] * Q[i, j, t] + s[i, j] * Z[i, j, t] + 
                            sum(r[j, k] * W[i, j, k, t] for k in J if k != j) for t in T) for j in J) for i in I)
m.setObjective(expr_objetivo, sense=GRB.MINIMIZE)

## Restrições

In [109]:
# Balanço de estoque (revisar comportamento)
# Período inicial
m.addConstrs((Q[i, j, t] == X[i, j, t] - sum(W[i, j, k, t] for k in J if k != j) + sum(W[i, l, j, t] for l in J if l != j) - d[i, j, t] for i in I for j in J for t in T if t == 0),
             name='restricao_balanco_estoque')
# Demais períodos
m.addConstrs((Q[i, j, t] == Q[i, j, t-1] + X[i, j, t] - sum(W[i, j, k, t] for k in J if k != j) + sum(W[i, l, j, t] for l in J if l != j) - d[i, j, t] for i in I for j in J for t in T if t > 0),
             name='restricao_balanco_estoque');

In [110]:
# Restrição que obriga setup (validar o range do r)
# m.addConstrs((X[i, j, t] <= min((cap[j, t] - f[i, j]) / b[i, j], sum(sum(d[i, k, r] for r in range(t, T[-1] + 1)) for k in J)) * Z[i, j, t] for i in I for j in J for t in T)
#              , name='restricao_setup');
m.addConstrs((X[i, j, t] <= sum(sum(d[i, j, b] for b in range(t, len(T))) for j in J) * Z[i, j, t] for i in I for j in J for t in T)
             , name='restricao_setup');

In [111]:
# Restrição de capacidade
m.addConstrs((sum(b[i, j] * X[i, j, t] + f[i, j] * Z[i, j, t] for i in I) <= cap[j, t] for j in J for t in T)
             , name='restricao_capacidade');

# Resolução

In [112]:
# Critérios de parada
m.Params.timelimit = 1800  # Tempo (default = inf)
# m.Params.MIPgap = 0.01  # Gap de otimalidade (default = 0.0001 = 0.01%)

# Otimização
m.optimize()

Set parameter TimeLimit to value 1800
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 624 rows, 2016 columns and 3720 nonzeros
Model fingerprint: 0x8852f4a5
Variable types: 1152 continuous, 864 integer (288 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [2e-01, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 0 rows and 288 columns
Presolve time: 0.01s
Presolved: 624 rows, 1728 columns, 3720 nonzeros
Variable types: 864 continuous, 864 integer (288 binary)

Root relaxation: objective 6.157549e+04, 1496 iterations, 0.03 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/

In [113]:
for i in I:
    for j in J:
        for k in J:
            for t in T:
                print(f'W[{i, j, k, t}] = {W[i, j, k, t].X}')

W[(0, 0, 0, 0)] = 0.0
W[(0, 0, 0, 1)] = 0.0
W[(0, 0, 0, 2)] = 0.0
W[(0, 0, 0, 3)] = 0.0
W[(0, 0, 0, 4)] = 0.0
W[(0, 0, 0, 5)] = 0.0
W[(0, 0, 0, 6)] = 0.0
W[(0, 0, 0, 7)] = 0.0
W[(0, 0, 0, 8)] = 0.0
W[(0, 0, 0, 9)] = 0.0
W[(0, 0, 0, 10)] = 0.0
W[(0, 0, 0, 11)] = 0.0
W[(0, 0, 1, 0)] = 103.0
W[(0, 0, 1, 1)] = 172.0
W[(0, 0, 1, 2)] = 90.0
W[(0, 0, 1, 3)] = 125.0
W[(0, 0, 1, 4)] = 52.0
W[(0, 0, 1, 5)] = 41.0
W[(0, 0, 1, 6)] = 108.0
W[(0, 0, 1, 7)] = 0.0
W[(0, 0, 1, 8)] = 164.0
W[(0, 0, 1, 9)] = 169.0
W[(0, 0, 1, 10)] = 126.0
W[(0, 0, 1, 11)] = 51.0
W[(0, 0, 2, 0)] = 101.0
W[(0, 0, 2, 1)] = 179.0
W[(0, 0, 2, 2)] = 154.0
W[(0, 0, 2, 3)] = 46.0
W[(0, 0, 2, 4)] = 94.0
W[(0, 0, 2, 5)] = 36.0
W[(0, 0, 2, 6)] = 153.0
W[(0, 0, 2, 7)] = 138.0
W[(0, 0, 2, 8)] = 41.0
W[(0, 0, 2, 9)] = 146.0
W[(0, 0, 2, 10)] = 33.0
W[(0, 0, 2, 11)] = 176.0
W[(0, 0, 3, 0)] = 116.0
W[(0, 0, 3, 1)] = 3.0
W[(0, 0, 3, 2)] = 117.0
W[(0, 0, 3, 3)] = 129.0
W[(0, 0, 3, 4)] = 12.0
W[(0, 0, 3, 5)] = 150.0
W[(0, 0, 3, 6)] = 76.0
W

In [114]:
# Número de períodos da instância
len(T)

12

In [115]:
# Número de fábricas
len(J)

4

In [116]:
# Número de produtos
len(I)

6

In [117]:
# Tempo de execução em segundos
m.Runtime

115.10500001907349

In [118]:
# Nome do modelo
m.ModelName

'Lot-sizing Sambasivan and Yahya'

In [119]:
# Valor da função objetivo da melhor solução encontrada (upper bound da minimização)
m.ObjVal

67139.87

In [120]:
# Lower bound da minimização
m.ObjBound

67139.87

In [121]:
# Gap da solução
m.MIPgap

0.0

In [122]:
m.Status  # Ver https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes para interpretar códigos

2